# We will apply some unsupervised techniques from lecture
## [scikit-learn](https://scikit-learn.org/stable/) has the most extensive machine learning algorithms and good documentation

## UMAP is implemented in a dedicated [umap-learn](https://umap-learn.readthedocs.io/en/latest/) package

## Visit these pages to get deeper understanding of [t-SNE](https://distill.pub/2016/misread-tsne/) and [UMAP](https://pair-code.github.io/understanding-umap/)

In [ ]:
# !pip install scikit-learn umap-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import umap

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, MDS

from scipy.spatial.distance import pdist, squareform

## Today's data is from [3,200 colorectal cancer patients](https://pubmed.ncbi.nlm.nih.gov/26457759/)
62 genes were selected from 6,000 to simplify the dataset

Mutation of key genes and microsatellite instability also included in the last columns

In [ ]:
data = pd.read_excel('CRC_62gene_full.xlsx', index_col = 0, header = 0)
data.head(3)

## Extract gene expression part

In [ ]:
first_mutation_column = list(data.columns).index('KRAS')

exp_data = data.iloc[:, :first_mutation_column]
exp_data.head(3)

# Dimensionality reduction
## Let's start with PCA
For PCA, data must be centered (each column has zero mean)

But it's optional whether to scale the data (each column has unit variance)

### First, PCA with centered data

In [ ]:
centered_data = exp_data - exp_data.mean()
print(centered_data.mean())

In [ ]:
centered_pca = PCA(random_state = 25).fit(centered_data)
centered_pca_embed = centered_pca.transform(centered_data)

### View amount of explained variance in each component

In [ ]:
plt.figure(figsize = (10, 4))
plt.subplot(1, 2, 1)
plt.bar(range(1, centered_pca.n_components_ + 1), centered_pca.explained_variance_ratio_)
plt.xlabel('PCA component')
plt.ylabel('explained variance ratio')

plt.subplot(1, 2, 2)
cumulative = np.cumsum(centered_pca.explained_variance_ratio_)
plt.bar(range(1, centered_pca.n_components_ + 1), cumulative)
plt.plot([1, exp_data.shape[1]], [0.95, 0.95], '--', color = 'tab:orange')
plt.xlabel('PCA component')
plt.ylabel('explained variance ratio')
plt.title('cumulative')

plt.tight_layout()
plt.show()

### Observation
* The first two components capture >25% of variance each
* About 38 components can capture 90% of the total variance

### View PCA embedding of the first two components
Color by some genes

In [ ]:
np.random.seed(25)
random_genes = np.random.choice(exp_data.columns, 4, replace = False)

In [ ]:
plt.figure(figsize = (12, 3))

for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.scatter(centered_pca_embed[:, 0], centered_pca_embed[:, 1], c = exp_data[random_genes[i]], 
                cmap = 'RdBu', s = 2)
    plt.xlabel('PC1'); plt.ylabel('PC2')
    plt.title(random_genes[i])

plt.tight_layout()
plt.show()

### View loadings on the first two components

In [ ]:
plt.figure(figsize = (13, 4))
plt.bar(range(exp_data.shape[1]), centered_pca.components_[0])
plt.xticks(range(exp_data.shape[1]), labels = exp_data.columns, rotation = 90)
plt.ylabel('PC1 loading')
plt.show()

plt.figure(figsize = (13, 4))
plt.bar(range(exp_data.shape[1]), centered_pca.components_[1])
plt.xticks(range(exp_data.shape[1]), labels = exp_data.columns, rotation = 90)
plt.ylabel('PC2 loading')
plt.show()

### Observation
* PC1 focuses on REG4, FCGBP, and MUC2
* PC2 has a lot of similar loadings, with the top three being SFRP2, REG4 (again), and GAS1

### Let's check the expression variance of these genes

In [ ]:
plt.figure(figsize = (13, 4))
plt.bar(range(exp_data.shape[1]), centered_data.var())
plt.xticks(range(exp_data.shape[1]), labels = exp_data.columns, rotation = 90)
plt.ylabel('Expression variance')
plt.show()

### Observation
* REG4, FCGBP, and MUC2 have the highest expression variances --> hence, PC1 focus on them

### Let's re-run PCA with standardized data instead

In [ ]:
std_data = (exp_data - exp_data.mean()) / exp_data.std()
print(std_data.std())

In [ ]:
std_pca = PCA(random_state = 25).fit(std_data)
std_pca_embed = std_pca.transform(std_data)

### Compare new loadings with previous one

In [ ]:
plt.figure(figsize = (13, 4))
plt.scatter(range(exp_data.shape[1]), centered_pca.components_[0], label = 'Centered')
plt.scatter(range(exp_data.shape[1]), std_pca.components_[0], label = 'Standardized')
plt.xticks(range(exp_data.shape[1]), labels = exp_data.columns, rotation = 90)
plt.ylabel('PC1 loading'); plt.legend()

plt.plot([0, exp_data.shape[1] - 1], [0, 0], 'k', alpha = 0.7)

### Add arrow
for i in range(exp_data.shape[1]):
    plt.plot([i, i], [centered_pca.components_[0][i], std_pca.components_[0][i]], 'k--')

plt.show()

### Observation
With standardization, PC1 now assigns similar weights to many genes instead of to ony 3-4 genes

### Check the new explained variance and embedding

In [ ]:
plt.figure(figsize = (10, 4))
plt.subplot(1, 2, 1)
plt.bar(range(1, std_pca.n_components_ + 1), std_pca.explained_variance_ratio_)
plt.xlabel('PCA component')
plt.ylabel('explained variance ratio')

plt.subplot(1, 2, 2)
cumulative = np.cumsum(std_pca.explained_variance_ratio_)
plt.bar(range(1, std_pca.n_components_ + 1), cumulative)
plt.plot([1, exp_data.shape[1]], [0.95, 0.95], '--', color = 'tab:orange')
plt.xlabel('PCA component')
plt.ylabel('explained variance ratio')
plt.title('cumulative')

plt.tight_layout()
plt.show()

In [ ]:
print('Centered data')
plt.figure(figsize = (12, 3))

for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.scatter(centered_pca_embed[:, 0], centered_pca_embed[:, 1], c = exp_data[random_genes[i]], cmap = 'RdBu', s = 2)
    plt.xlabel('PC1'); plt.ylabel('PC2')
    plt.title(random_genes[i])

plt.tight_layout()
plt.show()

print('---------------------------------------------------------------')
print('Standardized data')
plt.figure(figsize = (12, 3))

for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.scatter(std_pca_embed[:, 0], std_pca_embed[:, 1], c = exp_data[random_genes[i]], cmap = 'RdBu', s = 2)
    plt.xlabel('PC1'); plt.ylabel('PC2')
    plt.title(random_genes[i])

plt.tight_layout()
plt.show()

### Observation
* The embedding and explained variances are similar between the two method
* But the interpretation of genes are completely different

## To use non-Euclidean distance, we need to switch from PCA to MDS
MDS function in scikit-learn requires the non-Euclidean distance matrix be *precomputed*

**Important**: Correlation is a similarity function (higher = more similar), not a distance function (higher = less similar). Use **1 - correlation** instead.

In [ ]:
corr_matrix = exp_data.T.corr(method = 'pearson')
display(corr_matrix.head(3))

#### Select only the first 10000 patients to speed things up
Use *1 - abs(correlation)* as the distance

In [ ]:
samp_corr_matrix = corr_matrix.iloc[:1000, :1000]

In [ ]:
corr_mds = MDS(n_components = 2, random_state = 25, dissimilarity = 'precomputed')
corr_mds_embed = corr_mds.fit_transform(1 - np.abs(samp_corr_matrix))

In [ ]:
plt.figure(figsize = (12, 3))

for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.scatter(corr_mds_embed[:, 0], corr_mds_embed[:, 1], c = exp_data[random_genes[i]].iloc[:1000], 
                cmap = 'RdBu', s = 2)
    plt.xlabel('MDS1'); plt.ylabel('MDS2')
    plt.title(random_genes[i])

plt.tight_layout()
plt.show()

### Observation
* MDS with correlation generate a different pattern than PCA
* MDS took *quite a while* to fit the data

## t-SNE
We can use either standardized data + Euclidean or raw data + correlation

Try several perplexity values

Use **%%timeit** to measure the amount of CPU time used

In [ ]:
%%timeit -r 1 -n 1
perplexities = [5, 15, 25, 50, 100]

plt.figure(figsize = (13, 3))

for i, k in enumerate(perplexities, start = 1):
    plt.subplot(1, 5, i)
    tsne_embed = TSNE(n_components = 2, perplexity = k, random_state = 25).fit_transform(std_data.iloc[:1000, :1000])
    plt.scatter(tsne_embed[:, 0], tsne_embed[:, 1], s = 2)
    plt.xlabel('tSNE1'); plt.ylabel('tSNE2')
    plt.title('perplexity = ' + str(k))

plt.tight_layout()
plt.show()

## UMAP
Similar to t-SNE. Try several neighbor values

In [ ]:
%%timeit -r 1 -n 1 ## measure the amount of CPU time used
neighbers = [5, 15, 25, 50, 100]

plt.figure(figsize = (13, 3))

for i, n in enumerate(neighbers, start = 1):
    plt.subplot(1, 5, i)
    umap_embed = umap.UMAP(n_components = 2, n_neighbors = n, random_state = 25).fit_transform(std_data.iloc[:1000, :1000])
    plt.scatter(umap_embed[:, 0], umap_embed[:, 1], s = 2)
    plt.xlabel('UMAP1'); plt.ylabel('UMAP2')
    plt.title('n_neighbors = ' + str(n))

plt.tight_layout()
plt.show()

### Recalculate UMAP with n_neighbors = 25
UMAP can be run on the full dataset

In [ ]:
umap_embed = umap.UMAP(n_components = 2, n_neighbors = 25, random_state = 25).fit_transform(std_data)

In [ ]:
plt.figure(figsize = (12, 3))

for i in range(4):
    plt.subplot(1, 4, i + 1)
    plt.scatter(umap_embed[:, 0], umap_embed[:, 1], c = exp_data[random_genes[i]], 
                cmap = 'RdBu', s = 2)
    plt.xlabel('UMAP1'); plt.ylabel('UMAP2')
    plt.title(random_genes[i])

plt.tight_layout()
plt.show()

### Observation
* Consistent structure when perplexity / n_neighbors are 15 or higher
* Similar gradient of gene expressions as PCA

### Effect of min_dist

In [ ]:
min_dists = [0, 1]

plt.figure(figsize = (12, 3))

for i, m in enumerate(min_dists, start = 1):
    plt.subplot(1, 4, i)
    umap_embed = umap.UMAP(n_components = 2, n_neighbors = 25, min_dist = m, random_state = 25, 
                           metric = 'correlation').fit_transform(std_data)
    plt.scatter(umap_embed[:, 0], umap_embed[:, 1], s = 2)
    plt.xlabel('UMAP1'); plt.ylabel('UMAP2')
    plt.title('min_dist = ' + str(m))

plt.tight_layout()
plt.show()

### Finalize UMAP embedding to be used later on

In [ ]:
final_umap_embed = umap.UMAP(n_components = 2, n_neighbors = 25, min_dist = 0, 
                             random_state = 25, metric = 'correlation').fit_transform(std_data)

# Clustering with [scikit-learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster)

## Various [clustering scoring functions](https://scikit-learn.org/stable/modules/classes.html#clustering-metrics)

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score

## k-mean

In [ ]:
kmean_silhouette = []
kmean_caha = []

for k in range(2, 10):
    kmean = KMeans(n_clusters = k, random_state = 25, n_init = 5).fit_predict(std_data)
    kmean_silhouette.append(silhouette_score(std_data, kmean))
    kmean_caha.append(calinski_harabasz_score(std_data, kmean))

In [ ]:
plt.figure(figsize = (8, 3))
plt.subplot(1, 2, 1)
plt.plot(range(2, 10), kmean_silhouette)
plt.xlabel('Number of clusters'); plt.ylabel('Silhouette score')

plt.subplot(1, 2, 2)
plt.plot(range(2, 10), kmean_caha)
plt.xlabel('Number of clusters'); plt.ylabel('Calinski-Harabasz score')

plt.tight_layout()
plt.show()

### Visualize the location of the two clusters

In [ ]:
def view_clusters(labels):
    plt.figure(figsize = (10, 5))

    plt.subplot(1, 2, 1)

    for k in np.unique(labels):
        filt = labels == k
        plt.scatter(final_umap_embed[filt, 0], final_umap_embed[filt, 1], s = 2, label = 'Cluster ' + str(k))

    plt.xlabel('UMAP1'); plt.ylabel('UMAP2'); plt.legend()

    plt.subplot(1, 2, 2)

    for k in np.unique(labels):
        filt = labels == k
        plt.scatter(std_pca_embed[filt, 0], std_pca_embed[filt, 1], s = 2, label = 'Cluster ' + str(k))

    plt.xlabel('PC1'); plt.ylabel('PC2'); plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
predicted = KMeans(n_clusters = 2, random_state = 25).fit_predict(std_data)
view_clusters(predicted)

## Agglomerative / Hierarchical clustering 

In [ ]:
def evaluate_clustering(n_clusters, method, data = std_data):
    silhouette = []
    caha = []

    for k in n_clusters:
        method.n_clusters = k
        predicted = method.fit_predict(data)
        silhouette.append(silhouette_score(data, predicted))
        caha.append(calinski_harabasz_score(data, predicted))
        
    plt.figure(figsize = (8, 3))
    plt.subplot(1, 2, 1)
    plt.plot(range(2, 10), silhouette)
    plt.xlabel('Number of clusters'); plt.ylabel('Silhouette score')

    plt.subplot(1, 2, 2)
    plt.plot(range(2, 10), caha)
    plt.xlabel('Number of clusters'); plt.ylabel('Calinski-Harabasz score')

    plt.tight_layout()
    plt.show()

### Try with Euclidean distance

In [ ]:
evaluate_clustering(range(2, 10), AgglomerativeClustering(affinity = 'euclidean', linkage = 'average'))

In [ ]:
predicted = AgglomerativeClustering(n_clusters = 2, metric = 'euclidean', linkage = 'average').fit_predict(std_data)
view_clusters(predicted)

### Visualize hierarchical clustering with seaborn's [clustermap](https://seaborn.pydata.org/generated/seaborn.clustermap.html)

In [ ]:
_ = sns.clustermap(data = std_data, metric = 'euclidean', method = 'average', 
                   z_score = None, figsize = (6, 10), cmap = 'RdBu', center = 0, 
                   row_cluster = True, col_cluster = True, 
                   row_colors = None, col_colors = None)

### Observation
* There is a group of outliers in the dendrogram
* Hierarchical clustering with n_clusters = 2 simply distinguish these outliers from other patients

## DBSCAN doesn't need the number of cluster to be specified
But we need to tune **epsilon** instead

In [ ]:
start = 0.1
step = 0.05
eps = np.arange(start, step * 8 + start, step)

plt.figure(figsize = (12, 6))

for i, e in enumerate(eps, start = 1):
    plt.subplot(2, 4, i)
    dbscan = DBSCAN(eps = e, metric = 'correlation', min_samples = 10)
    predicted = dbscan.fit_predict(std_data)
    
    for k in sorted(np.unique(predicted)):
        filt = predicted == k
        plt.scatter(final_umap_embed[filt, 0], final_umap_embed[filt, 1], label = 'cluster ' + str(k), s = 2)

    plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.legend()
    plt.title('epsilon = ' + str(e)[:4])

plt.tight_layout()
plt.show()    

### Observation
* DBSCAN is a density-based technique
* Since the key structure here is continuous, DBSCAN detect only a single dense cluster

## Lastly, network clustering
[Louvain and Leiden](https://www.nature.com/articles/s41598-019-41695-z) algorithms

In [ ]:
# !pip install --upgrade python-louvain networkx

In [ ]:
import networkx as nx
import community

## Create a network of absolute correlation between patients
### Remove edges with correlation less than 0.7 to simplify the data
Don't worry about the details. [Networkx](https://networkx.org/) commands are beyond the scope of this course

In [ ]:
thresholded_corr_matrix = exp_data.T.corr(method = 'pearson').to_numpy()
thresholded_corr_matrix[thresholded_corr_matrix < 0.7] = 0
thresholded_corr_matrix -= np.eye(data.shape[0]) ## remove diagonal entries
    
correlation_network = nx.from_numpy_array(thresholded_corr_matrix)
correlation_network = nx.relabel_nodes(correlation_network, lambda x: data.index[x]) ## Add patient names to nodes
_ = correlation_network.edges(data = True)

### Visualize a random subnetwork

In [ ]:
np.random.seed(25)
selected_samples = np.random.choice(data.index, size = 300, replace = False)
selected_samples = max(nx.connected_components(correlation_network.subgraph(selected_samples)), key = len)
selected_graph = correlation_network.subgraph(selected_samples)
_ = selected_graph.edges(data = True)

plt.figure(figsize = (8, 8))
edges, weights = zip(*nx.get_edge_attributes(selected_graph, 'weight').items())
nx.draw_spring(selected_graph, node_size = 40, edgelist = edges, edge_color = weights, edge_cmap = plt.cm.Greys, alpha = 0.8)
plt.show()

## Use Louvain algorithm to identify partitions that maximize modularity
Partition is a dictionary that map node name to cluster ID

In [ ]:
partition = community.best_partition(correlation_network, random_state = 25)

In [ ]:
predicted = np.array([partition['Patient' + str(x)] for x in range(1, data.shape[0] + 1)])
unique, counts = np.unique(predicted, return_counts = True)

plt.figure(figsize = (5, 3))
plt.hist(counts, bins = 20)
plt.xlabel('Partition size'); plt.ylabel('Number of nodes')
plt.show()

### Observation
* There are a few outliers + 3 large clusters

### Visualize only large partitions

In [ ]:
def view_large_clusters(labels, size_cutoff):
    outliers = []
    
    plt.figure(figsize = (10, 5))
    plt.subplot(1, 2, 1)

    for k in np.unique(labels):
        filt = labels == k
        
        if filt.sum() >= size_cutoff:
            plt.scatter(final_umap_embed[filt, 0], final_umap_embed[filt, 1], s = 2, label = 'Cluster ' + str(k))
        else:
            outliers.append(k)

    filt = [label for label in labels if label in outliers]
    plt.scatter(final_umap_embed[filt, 0], final_umap_embed[filt, 1], s = 2, label = 'Outliers', c = 'tab:gray', alpha = 0.3)       
    plt.xlabel('UMAP1'); plt.ylabel('UMAP2'); plt.legend()

    plt.subplot(1, 2, 2)

    for k in np.unique(labels):
        filt = labels == k
        
        if filt.sum() >= size_cutoff:
            plt.scatter(std_pca_embed[filt, 0], std_pca_embed[filt, 1], s = 2, label = 'Cluster ' + str(k))

    filt = [label for label in labels if label in outliers]
    plt.scatter(std_pca_embed[filt, 0], std_pca_embed[filt, 1], s = 2, label = 'Outliers', c = 'tab:gray', alpha = 0.3)
    plt.xlabel('PC1'); plt.ylabel('PC2'); plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
view_large_clusters(predicted, 20)

## Show these clusters on clustermap

In [ ]:
cluster_cmap = {1: 'tab:blue', 3: 'tab:orange', 4: 'tab:green'}
louvain_cluster_color = []

for x in predicted:
    if x in cluster_cmap:
        louvain_cluster_color.append(cluster_cmap[x])
    else: ## outliers
        louvain_cluster_color.append('tab:gray')
        
_ = sns.clustermap(data = std_data, metric = 'correlation', method = 'average', 
                   z_score = None, figsize = (6, 10), cmap = 'RdBu', center = 0, 
                   row_cluster = True, col_cluster = True, 
                   row_colors = louvain_cluster_color, col_colors = None)

### Observation
* Good agreement between Louvain network clustering and clustermap's dendrogram
* Some cluster contain sub-clusters

## Use mutation data to validate cluster

In [ ]:
mutation_cmap = {'wt': 'tab:cyan', 'mt': 'tab:red', 'MSS': 'tab:cyan', 'MSI': 'tab:red'}

In [ ]:
plt.figure(figsize = (12, 8))

plt.subplot(2, 3, 1)
outliers = []
    
for k in np.unique(predicted):
    filt = predicted == k

    if filt.sum() >= 20:
        plt.scatter(final_umap_embed[filt, 0], final_umap_embed[filt, 1], s = 4, 
                    label = 'Cluster ' + str(k), alpha = 0.6)
    else:
        outliers.append(k)

filt = [p for p in predicted if p in outliers]
plt.scatter(final_umap_embed[filt, 0], final_umap_embed[filt, 1], s = 4, label = 'Outliers', 
            c = 'tab:gray', alpha = 0.1)       

plt.xlabel('UMAP1'); plt.ylabel('UMAP2'); plt.legend()

for i, mutation in enumerate(data.columns[first_mutation_column:], start = 2):
    plt.subplot(2, 3, i)

    for m in pd.unique(data[mutation]):
        if not pd.isna(m):
            filt = data[mutation] == m
            plt.scatter(final_umap_embed[filt, 0], final_umap_embed[filt, 1], s = 4, label = m,
                        marker = 'x', c = mutation_cmap[m])       

    plt.xlabel('UMAP1'); plt.ylabel('UMAP2')
    plt.title(mutation)

plt.tight_layout()
plt.show()

### Add cluster label to data and summarize frequency

In [ ]:
mutation_rate = pd.DataFrame(0, index = [1, 3, 4],  
                             columns = data.columns[first_mutation_column:])

for m in mutation_rate.columns:
    for c in mutation_rate.index:
        wt_count = (((data[m] == 'wt') | (data[m] == 'MSS')) & (predicted == c)).sum()
        mt_count = (((data[m] == 'mt') | (data[m] == 'MSI')) & (predicted == c)).sum()
        mutation_rate.loc[c, m] = mt_count / (wt_count + mt_count)
        
mutation_rate.head()

### Observation
* Cluster 4 is associated with high BRAF, high MSI, low APC, and low TP53